# Pytorch Baseline - Train

**Notes**
- Do not forget to enable the GPU (TPU) for training
- You have to add `kaggle_l5kit` as utility script
- Parts of the code below is from the [official example](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb)
- [Baseline inference notebook](https://www.kaggle.com/pestipeti/pytorch-baseline-inference)

In [1]:
import numpy as np
from numpy import ma
import os
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18,resnet50,resnet101
from tqdm import tqdm
from typing import Dict

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

from pykalman import AdditiveUnscentedKalmanFilter
from joblib import Parallel, delayed
import math
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv

In [2]:
DIR_INPUT = "/media/ubuntu/Data/project/lyft/lyft-motion-prediction-autonomous-vehicles/"

In [3]:
cfg = {
    'format_version': 4,
    'model_params': {
        'history_num_frames': 100,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [1, 1],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 0
    }
}

In [4]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

## Dataset, dataloader

In [5]:
torch.cuda.is_available()

True

In [6]:
rasterizer = build_rasterizer(cfg, dm)

test_zarr = ChunkedDataset(dm.require(cfg['test_data_loader']["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset, 
                             shuffle=False, 
                             batch_size=cfg['test_data_loader']["batch_size"], 
                             num_workers=0)

warning, you're running with a custom agents_mask


In [7]:
def MotionModel(currentState,ang_rng=None):
    nextState    = np.zeros(6)
    nextState[0] = currentState[0] +  np.cos(currentState[3]) *  currentState[2]# * deltaTimeStamp + 0.5 * currentState[5]* (deltaTimeStamp**2) ) 
    nextState[1] = currentState[1] +  np.sin(currentState[3]) *  currentState[2] # * deltaTimeStamp + 0.5 * currentState[5]* (deltaTimeStamp**2) ) 
    nextState[2] = currentState[2] +  currentState[5] 
    nextState[3] = currentState[3] +  currentState[4]                                                                                                            
    if ang_rng is not None:
        nextState[3] = np.clip(nextState[3], ang_rng[0], ang_rng[1])                                                                
    nextState[4] = currentState[4]                                                                                                                     
    nextState[5] = currentState[5]  * 0.9                                                                                                                    
    return nextState

def MeasurementModel(currentState):
    measureState = np.zeros(3)
    measureState[0] = currentState[0]
    measureState[1] = currentState[1]
    measureState[2] = currentState[3]
    return measureState

def f(cs, ang_rng=None):
    res = np.zeros(6)
    res[0] = cs[0] + cs[2]*np.cos(cs[3])
    res[1] = cs[1] + cs[2]*np.sin(cs[3])
    res[2] = cs[2] + cs[5]
    res[3] = cs[3] + cs[4]
    if ang_rng is not None:
        res[3] = np.clip(res[3], ang_rng[0], ang_rng[1])
    res[4] = cs[4]
    res[5] = 0.9*cs[5]
    return res

def g(cs):
    res = np.zeros(2)
    res[0] = cs[0]
    res[1] = cs[1]
    return res

timestamps = []
agent_ids = []
future_coords_offsets_pd = []

for batch_idx, data in enumerate(tqdm(test_dataloader)):
    
    history_positions = data['history_positions'].cpu().numpy()
    history_yaws = data['history_yaws'].cpu().numpy()
    history_availabilities = data['history_availabilities'].cpu().numpy()
    timestamp = data["timestamp"].cpu().numpy()
    track_id = data["track_id"].cpu().numpy()
    
    def KalmanfilterExec(historyPostion,historyYaw,historyAvailability,timeStamp,trackId):

        measurements = historyPostion[::-1]
        #measurements = np.hstack( (measurements,historyYaw[::-1] )) 

        ang_std = 0.01
        Q = 0.001*np.diag([1, 1,ang_std**2, ang_std**2, 1, 0.001])
        m0 = measurements[-1]

        kf = AdditiveUnscentedKalmanFilter(initial_state_mean = [m0[0],m0[1],0,0,0,0], 
                                           n_dim_obs=2,
                                           transition_functions = MotionModel,
                                           observation_functions = MeasurementModel,
                                           transition_covariance = Q,
                                           initial_state_covariance = Q,
                                           observation_covariance = 0.1**2*np.eye(2))

        previousSate = ma.array(measurements)
        previousSate[historyAvailability[::-1] < 0.5] = ma.masked

        z = kf.smooth(previousSate)

        prediction = np.zeros((51,6))
        prediction[0] = z[0][-1]
        ang_rng = (z[0][-10:,3].min() - math.pi/3, z[0][-10:,3].max() + math.pi/3)
        for i in range(1,51):
            prediction[i] = MotionModel(prediction[i-1], ang_rng)
        prediction = prediction[1:,:2]
        
        return timeStamp, trackId, np.expand_dims(prediction,0)

    res = Parallel(n_jobs=4)(delayed(KalmanfilterExec)(history_positions[i], history_yaws[i],history_availabilities[i], 
                                          timestamp[i], track_id[i]) for i in range(len(data['history_positions'])))
    
    timestamps.append(np.stack([r[0] for r in res]))
    agent_ids.append(np.stack([r[1] for r in res]))
    future_coords_offsets_pd.append(np.concatenate([r[2] for r in res]))

print(np.concatenate(future_coords_offsets_pd).shape)
write_pred_csv("/media/ubuntu/Data/project/lyft/l5kit-1.0.6/examples/agent_motion_prediction/submission_KF_new.csv",
       timestamps=np.concatenate(timestamps),
       track_ids=np.concatenate(agent_ids),
       coords=np.concatenate(future_coords_offsets_pd),
      )

In [8]:
timestamps = []
agent_ids = []
future_coords_offsets_pd = []

for batch_idx, data in enumerate(tqdm(test_dataloader)):
    
    history_positions = data['history_positions'].cpu().numpy()
    history_yaws = data['history_yaws'].cpu().numpy()
    history_availabilities = data['history_availabilities'].cpu().numpy()
    timestamp = data["timestamp"].cpu().numpy()
    track_id = data["track_id"].cpu().numpy()
    
    def run(hp,hy,ha,ts,ti):

        measurements = hp[::-1]
        measurements = np.hstack( (measurements,hy[::-1] )) 

        ang_std = 0.01
        Q = 0.001*np.diag([1, 1, 1, ang_std**2, ang_std**2, 0.001])
        m0 = measurements[-1]

        kf = AdditiveUnscentedKalmanFilter(initial_state_mean = [m0[0],m0[1],0,m0[2],0,0], 
                                           n_dim_obs=3,
                                           transition_functions = MotionModel,
                                           observation_functions = MeasurementModel,
                                           transition_covariance = Q,
                                           initial_state_covariance = Q,
                                           observation_covariance = 0.1**2*np.eye(3))

        X = ma.array(measurements)
        X[ha[::-1] < 0.5] = ma.masked

        z = kf.smooth(X)

        pred = np.zeros((51,6))
        pred[0] = z[0][-1]
        ang_rng = (z[0][-10:,3].min() - math.pi/3, z[0][-10:,3].max() + math.pi/3)
        for i in range(1,51):
            pred[i] = MotionModel(pred[i-1], ang_rng)
        pred = pred[1:,:2]
        
        return ts, ti, np.expand_dims(pred,0)

    res = Parallel(n_jobs=4)(delayed(run)(history_positions[i], history_yaws[i],history_availabilities[i], 
                                          timestamp[i], track_id[i]) for i in range(len(data['history_positions'])))
    
    timestamps.append(np.stack([r[0] for r in res]))
    agent_ids.append(np.stack([r[1] for r in res]))
    future_coords_offsets_pd.append(np.concatenate([r[2] for r in res]))

print(np.concatenate(future_coords_offsets_pd).shape)
write_pred_csv("/media/ubuntu/Data/project/lyft/l5kit-1.0.6/examples/agent_motion_prediction/submission_KF_1.csv",
       timestamps=np.concatenate(timestamps),
       track_ids=np.concatenate(agent_ids),
       coords=np.concatenate(future_coords_offsets_pd),
      )

100%|██████████| 8891/8891 [3:53:07<00:00,  1.57s/it]  


(71122, 50, 2)
